In [1]:
import os
import glob
import numpy as np
from pint.models import get_model
import pint.logging
from astropy import units as u
from astropy.coordinates import SkyCoord, ICRS
from astropy import constants as const

pint.logging.setup(level="WARNING")

1

In [2]:
def deg_GBT(sep,cfreq,diam=100.0 * u.m):
    """ Calculate degradation given angular separation, center frequency
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. The default diameter is for the GBT and the beam's full width at
    half max is calculated using FWHM = 1.2*lamda/eff_diam rad.
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    cfreq: quantity, center frequency of observation (MHz)
    diam: optional quantity, effective diameter of telescope (meters)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    lamda = const.c/cfreq
    fwhm = 1.2*(lamda/diam*u.rad).to(u.arcmin)
    sig = fwhm/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [3]:
def deg_AO(sep,cfreq,diam=300.0 * u.m):
    """ Calculate degradation given angular separation, center frequency
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. The default diameter is for AO and the beam's full width at
    half max is calculated using FWHM = 1.2*lamda/eff_diam rad.
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    cfreq: quantity, center frequency of observation (MHz)
    diam: optional quantity, effective diameter of telescope (meters)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    lamda = const.c/cfreq
    fwhm = 1.2*(lamda/diam*u.rad).to(u.arcmin)
    sig = fwhm/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [4]:
def deg_LOFAR(sep,fwhm=5 * u.arcmin):
    """ Calculate degradation given angular separation
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. 
    
    The default FWHM is 5 arcmin: https://www.astron.nl/lofarscience2013/presentations/Session2_hessels.pdf page 8
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    fwhm: optional quantity, full-width half-maximum of the telescope beam (arcmin)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    sig = fwhm.to(u.arcmin)/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [5]:
def deg_LWA1(sep,fwhm=2 * u.deg):
    """ Calculate degradation given angular separation
    
    This calculation assumes the beam's sensitivity pattern is well-approximated by a
    Gaussian function. 
    
    The default FWHM is 2 degrees, the value for the very top of the usable LWA1 band 
    around 80 MHz (Taylor et al. 2012)
    
    Parameters
    ==========
    sep: quantity, separation between observed/actual positions (arcmin)
    fwhm: optional quantity, full-width half-maximum of the telescope beam (degrees)
    
    Returns
    =======
    deg: float, estimated degradation to sensitivity based on separation
    """
    sig = fwhm.to(u.arcmin)/2.35482 # (solve for sigma when amplitude is 1/2; see wiki)
    
    return np.exp(-(sep/sig)**2/2)

In [6]:
MHz = 1.0e6*u.Hz
pars = np.sort(glob.glob("*[0-9]_fiore+22.par"))

In [15]:
print(deg_LOFAR(0.98*u.arcmin))

0.8989644724293145


In [18]:
data_dir = "./"
deg_factors = []
psrsfreqs = []
pars = sorted(glob.glob(f"{data_dir}*fiore+23.par"))

for p in pars:
    par_path = os.path.join(data_dir, p)
        
    # For each scan, get the RA/Dec and compare to timing position (eqcoord) 
    with open(p, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("PSRJ"):
                psr = l.split()[1]
            elif l.startswith("RAJ"):
                ra_str = l.split()[1]
            elif l.startswith("DECJ"):
                dec_str = l.split()[1].strip("+")
            else:
                pass
    
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))
    
    mlfrd_path = os.path.join(data_dir,f"{psr}.mlfrd")
    
    # mfrd files contain MJD, length, center frequency, RA, Dec info for each scan for a given pulsar
    with open(mlfrd_path) as f:
        mlfrd_lines = f.readlines()
    
    fnames = []
    freqs = []
    obstime = 0
    separations = [] # gather angular separations (arcmins)
    degradations = [] # and calculate degradations based on telescope
    lengths = [] # used to weight degradation factor later
    for ml in mlfrd_lines:
        if ml.startswith(('GUPPI', 'guppi', 'VEGAS')):
            fname, mjd_str, len_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freq = float(freq_str)
            sep = eqcoord.separation(c).arcminute
            deg = deg_GBT(sep*u.arcmin,freq*MHz)
            freqs.append(freq)
            lengths.append(float(len_str))
            separations.append(sep)
            degradations.append(deg)
            obstime += float(len_str)
        elif ml.startswith(('puppi')):
            fname, mjd_str, len_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freq = float(freq_str)
            sep = eqcoord.separation(c).arcminute
            deg = deg_AO(sep*u.arcmin,freq*MHz)
            freqs.append(freq)
            lengths.append(float(len_str))
            separations.append(sep)
            degradations.append(deg)
            obstime += float(len_str)
        elif ml.startswith(('J1327')):
            fname, mjd_str, len_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freq = float(freq_str)
            sep = eqcoord.separation(c).arcminute
            deg = deg_LWA1(sep*u.arcmin)
            freqs.append(freq)
            lengths.append(float(len_str))
            separations.append(sep)
            degradations.append(deg)
            obstime += float(len_str)
        elif ml.startswith(('J0215')):
            fname, mjd_str, len_str, freq_str, ra_str, dec_str = ml.strip().split()
            c = SkyCoord(ra_str, dec_str, frame='icrs', unit=(u.hourangle, u.deg))
            fnames.append(fname)
            freq = float(freq_str)
            sep = eqcoord.separation(c).arcminute
            deg = deg_LOFAR(sep*u.arcmin)
            freqs.append(freq)
            lengths.append(float(len_str))
            separations.append(sep)
            degradations.append(deg)
            obstime += float(len_str)
    
    # Closer look at separations
    lengths = np.array(separations)
    separations = np.array(separations)
    degradations = np.array(degradations)
    n2000,n1500,n1430,n1380,n820,n430,n350,n149,n72,n57,n42 = freqs.count(2000.0),freqs.count(1500.0),freqs.count(1430.0),freqs.count(1380.0),freqs.count(820.0),freqs.count(430.0),freqs.count(350.0),freqs.count(148.926),freqs.count(71.85),freqs.count(57.15),freqs.count(42.45)
    nother = len(freqs) - (n42+n57+n72+n149+n350+n430+n820+n1380+n1430+n1500+n2000)
    print(f"PSR {psr} was observed {len(separations)} times")
    
    PRINT_FILES = True
    for ff in [42.45,57.15,71.85,148.926,350.0,430.0,820.0,1380.0,1430.0,1500.0,2000.0]:
        i_freq = np.where(np.array(freqs) == ff)[0]
        # separation thresholds (st) chosen to limit degradation to 10% when possible
        if ff == 42.45:
            st = 56.8 # arcmin
            nt = n42
        if ff == 57.15:
            st = 42.2
            nt = n57
        if ff == 71.85:
            st = 33.5
            nt = n72
        if ff == 148.926:
            st = 0.98
            nt = n149
        if ff == 350.0:
            if psr == "J0415+6111":
                st = 11.0 #only detection
            elif psr == "J1505-2524":
                st = 15.2 #increase threshold to 40%
            elif psr == "J1530-2114":
                st = 10.1 #increase threshold to 20%
            elif psr == "J1913+3732":
                st = 29.3 #increase threshold to 90%
            elif psr == "J1929+6630":
                st = 23.3 #increase threshold to 70%
            elif psr == "J1930+6205":
                st = 15.2 #increase threshold to 40%
            elif psr == "J2104+2830":
                st = 23.3 #increase threshold to 70%
            elif psr == "J2115+6702":
                st = 9.0 #only detection
            elif psr == "J2145+2158":
                st = 16.0 #only detection
            elif psr == "J2210+5712":
                st = 18.0 #only detection
            else:
                st = 7.0
            nt = n350
        if ff == 430.0:
            st = 5.7
            nt = n430
        if ff == 820.0:
            if psr == "J0141+6303":
                st = 7.6 #increase threshold to 50%
            elif psr == "J0415+6111":
                st = 4.9 #increase threshold to 25%
            elif psr == "J1530-2114":
                st = 5.9 #increase threshold to 30%
            elif psr == "J1913+3732":
                st = 13.8 #increase threshold to 90%
            elif psr == "J1929+6630":
                st = 4.3 #increase threshold to 20%
            elif psr == "J1930+6205":
                st = 4.9 #increase threshold to 25%
            elif psr == "J2104+2830":
                st = 4.9 #increase threshold to 25%
            elif psr == "J2210+5712":
                st = 4.3 #increase threshold to 20%
            else:
                st = 3.0
            nt = n820
        if ff == 1380.0:
            st = 1.7
            nt = n1380
        if ff == 1430.0:
            st = 1.7
            nt = n1430
        if ff == 1500.0:
            st = 1.7
            nt = n1500
        if ff == 2000.0:
            st = 1.3
            nt = n2000
        nws = len(np.where(separations[i_freq]<st)[0]) # number within separation threshold
        if separations[i_freq].any():
            print(
            f"  {nws} time(s) at {ff:.0f} MHz within {st:.1f} arcmin of the timing position ({nt} total)"
            )
            minsep = np.min(separations[i_freq])
            maxsep = np.max(separations[i_freq])
            medsep = np.median(separations[i_freq])
            
            freq_cut = [(f==ff) for f in freqs]
            seps_cut = (separations < st)
            weights = lengths[freq_cut*seps_cut]
            
            if ff < 100.0:
                mindeg = deg_LWA1(minsep*u.arcmin)
                maxdeg = deg_LWA1(maxsep*u.arcmin)
                meddeg = deg_LWA1(medsep*u.arcmin)
                degs   = deg_LWA1(separations[freq_cut*seps_cut]*u.arcmin)
            elif ff == 148.926:
                mindeg = deg_LOFAR(minsep*u.arcmin)
                maxdeg = deg_LOFAR(maxsep*u.arcmin)
                meddeg = deg_LOFAR(medsep*u.arcmin)
                degs   = deg_LOFAR(separations[freq_cut*seps_cut]*u.arcmin)
            elif ff == 327.0 or ff == 430.0 or 1000. < ff < 1450.0:
                mindeg = deg_AO(minsep*u.arcmin,ff*MHz)
                maxdeg = deg_AO(maxsep*u.arcmin,ff*MHz)
                meddeg = deg_AO(medsep*u.arcmin,ff*MHz)
                degs   = deg_AO(separations[freq_cut*seps_cut]*u.arcmin,ff*MHz)
            else:
                mindeg = deg_GBT(minsep*u.arcmin,ff*MHz)
                maxdeg = deg_GBT(maxsep*u.arcmin,ff*MHz)
                meddeg = deg_GBT(medsep*u.arcmin,ff*MHz)
                degs   = deg_GBT(separations[freq_cut*seps_cut]*u.arcmin,ff*MHz)
            
            deg_final = np.average(degs,weights=np.sqrt(weights)) # degradation of accepted files, weighted by sqrt(tobs)
            
            freq=int(ff)
            if ff==42.45 or ff==71.85 or ff==1430:
                pass
            else:
                psrsfreqs.append(f"{psr}   {int(round(freq,0))}")
                deg_factors.append(deg_final)
            
            print(f"  Min separation at {ff:.0f} MHz: {minsep:.1f} arcmin (deg = {mindeg:0.2f})")
            print(f"  Med separation at {ff:.0f} MHz: {medsep:.1f} arcmin (deg = {meddeg:0.2f})")
            print(f"  Max separation at {ff:.0f} MHz: {maxsep:.1f} arcmin (deg = {maxdeg:0.2f})")
            print(f"  Max separation at {ff:.0f} MHz: {maxsep:.1f} arcmin (deg = {maxdeg:0.2f})")
            print(f"  Degradation factor at {ff:.0f} MHz: {deg_final:.2f}")
        else:
            if ff == 350.0 or ff == 820.0:
                print(f"  (No detections at {ff:.0f} MHz)")
            
        if PRINT_FILES:
            sum_fname = f"{psr}_{int(ff)}.files"
            #print(np.array(fnames)[freq_cut*seps_cut])
            if nws != nt:
                with open(sum_fname,'w') as f:
                    for file in np.array(fnames)[freq_cut*seps_cut]:
                        f.write(file+'\n')
    #print(f"  PSR {psr} has {nother} observations at unaccounted-for frequencies")
if PRINT_FILES and len(psrsfreqs)==len(deg_factors):
    fname = "deg_factors.dat"
    with open(fname, 'w') as f:
        f.write("   PSR      FREQ (MHz)   DEG\n")
        for psrfreq,df in zip(psrsfreqs,deg_factors):
            psr = psrfreq.split()[0]
            freq = psrfreq.split()[1]
            if freq == "57":
                freq = "57.15"
            f.write(f"{psr}     {freq}      {float(df):.2f}\n")

PSR J0032+6946 was observed 81 times
  13 time(s) at 350 MHz within 7.0 arcmin of the timing position (42 total)
  Min separation at 350 MHz: 3.3 arcmin (deg = 0.98)
  Med separation at 350 MHz: 8.4 arcmin (deg = 0.85)
  Max separation at 350 MHz: 26.7 arcmin (deg = 0.21)
  Max separation at 350 MHz: 26.7 arcmin (deg = 0.21)
  Degradation factor at 350 MHz: 0.97
  13 time(s) at 820 MHz within 3.0 arcmin of the timing position (39 total)
  Min separation at 820 MHz: 0.0 arcmin (deg = 1.00)
  Med separation at 820 MHz: 3.8 arcmin (deg = 0.84)
  Max separation at 820 MHz: 14.5 arcmin (deg = 0.08)
  Max separation at 820 MHz: 14.5 arcmin (deg = 0.08)
  Degradation factor at 820 MHz: 1.00
PSR J0141+6303 was observed 37 times
  16 time(s) at 350 MHz within 7.0 arcmin of the timing position (17 total)
  Min separation at 350 MHz: 0.0 arcmin (deg = 1.00)
  Med separation at 350 MHz: 6.5 arcmin (deg = 0.91)
  Max separation at 350 MHz: 7.9 arcmin (deg = 0.87)
  Max separation at 350 MHz: 7.9 ar

  Min separation at 350 MHz: 9.7 arcmin (deg = 0.81)
  Med separation at 350 MHz: 9.7 arcmin (deg = 0.81)
  Max separation at 350 MHz: 20.6 arcmin (deg = 0.39)
  Max separation at 350 MHz: 20.6 arcmin (deg = 0.39)
  Degradation factor at 350 MHz: 0.81
  21 time(s) at 820 MHz within 5.9 arcmin of the timing position (27 total)
  Min separation at 820 MHz: 3.1 arcmin (deg = 0.89)
  Med separation at 820 MHz: 5.4 arcmin (deg = 0.71)
  Max separation at 820 MHz: 25.1 arcmin (deg = 0.00)
  Max separation at 820 MHz: 25.1 arcmin (deg = 0.00)
  Degradation factor at 820 MHz: 0.73
PSR J1816+4510 was observed 93 times
  23 time(s) at 350 MHz within 7.0 arcmin of the timing position (29 total)
  Min separation at 350 MHz: 0.0 arcmin (deg = 1.00)
  Med separation at 350 MHz: 1.1 arcmin (deg = 1.00)
  Max separation at 350 MHz: 26.6 arcmin (deg = 0.21)
  Max separation at 350 MHz: 26.6 arcmin (deg = 0.21)
  Degradation factor at 350 MHz: 1.00
  38 time(s) at 820 MHz within 3.0 arcmin of the timing

In [73]:
J0957_ra  = "09:57:08.1229793"
J0957_dec = "-06:19:37.50795"
J0957_pos = SkyCoord(J0957_ra,J0957_dec,frame=ICRS,unit=(u.hourangle, u.deg))

In [74]:
ras = ["09:55:07.848",
       "09:57:52.176",
       "09:56:10.176",
       "09:58:57.456",
       "09:54:28.920",
       "09:57:10.944",
       "09:59:56.976",
       "09:55:29.976",
       "09:58:16.872"]
decs = ["-05:28:07.320",
        "-05:39:08.280",
        "-05:56:08.880",
        "-06:05:03.120",
        "-06:13:40.800",
        "-06:23:32.280",
        "-06:34:32.160",
        "-06:41:43.440",
        "-06:51:40.680"]

In [82]:
for r,d in zip(ras, decs):
    pointing = SkyCoord(r,d,frame=ICRS,unit=(u.hourangle, u.deg))
    sep = pointing.separation(J0957_pos).to(u.arcmin)
    deg = deg_GBT(sep,350*u.MHz)
    print(pointing.ra.hms,pointing.dec,sep.value,deg)

hms_tuple(h=9.0, m=55.0, s=7.847999999994926) -5d28m07.32s 59.557856201003815 0.0003794580306933756
hms_tuple(h=9.0, m=57.0, s=52.176000000006724) -5d39m08.28s 41.94255033318454 0.020112267008510362
hms_tuple(h=9.0, m=56.0, s=10.176000000003569) -5d56m08.88s 27.5435377680924 0.18550962404225455
hms_tuple(h=9.0, m=58.0, s=57.45600000000593) -6d05m03.12s 30.83426473383106 0.12108804885714836
hms_tuple(h=9.0, m=54.0, s=28.92000000000209) -6d13m40.8s 40.00629152388854 0.02860785693119088
hms_tuple(h=9.0, m=57.0, s=10.943999999998795) -6d23m32.28s 3.9751495560854453 0.9655190275878642
hms_tuple(h=9.0, m=59.0, s=56.975999999999374) -6d34m32.16s 44.517335466564326 0.012267950399368002
hms_tuple(h=9.0, m=55.0, s=29.975999999999203) -6d41m43.44s 32.90391709119615 0.09034050121228483
hms_tuple(h=9.0, m=58.0, s=16.871999999999048) -6d51m40.68s 36.3165288250481 0.05346518822501614
